# Welcome to cfnet

> A fast and scalable library for counterfactual explanations in Jax.

## Key Features

- **fast**: code runs significantly faster than existing CF explanation libraries.
- **scalable**: code can be accelerated over *CPU*, *GPU*, and *TPU*
- **flexible**: we provide flexible API for researchers to allow full customization.


TODO: 
- implement various methods of CF explanations


## Install

`cfnet` is built on top of [Jax](https://jax.readthedocs.io/en/latest/index.html). It also uses [Pytorch](https://pytorch.org/) to load data.

### Running on CPU

If you only need to run `cfnet` on CPU, you can simply install via `pip` or clone the `GitHub` project.

Installation via PyPI:
```bash
pip install cfnet
```

Editable Install:
```bash
git clone https://github.com/BirkhoffG/cfnet.git
pip install -e cfnet
```

### Running on GPU or TPU

If you wish to run `cfnet` on GPU or TPU, please first install this library via `pip install cfnet`.

Then, you should install the right GPU or TPU version of Jax by following steps in the [install guidelines](https://github.com/google/jax#installation).



## An Example of using `cfnet`

See [How to use cfnet](tutorials/how_to_use_cfnet.ipynb).